# Potential Local Authorities (Motorways)

## Imports

In [23]:
import urllib.request, json
import pandas as pd
import math
import pickle
import collections
from tqdm import tqdm
import geopandas
import datetime
import os
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

## Global Variables

In [24]:
UNWANTED_COLS = ['region_name', 'region_id', 'road_type', 'pedal_cycles', 'two_wheeled_motor_vehicles',
                    'cars_and_taxis', 'buses_and_coaches', 'lgvs', 'hgvs_2_rigid_axle',
                    'hgvs_3_rigid_axle', 'hgvs_4_or_more_rigid_axle',
                    'hgvs_3_or_4_articulated_axle', 'hgvs_5_articulated_axle',
                    'hgvs_6_articulated_axle', 'all_hgvs', 'start_junction_road_name', 'end_junction_road_name', 'easting',
                    'northing', 'estimation_method', 'estimation_method_detailed', 'link_length_miles']


AADT_LOCAL_AUTHORITY_IDS = [('Luton', '120'), ('Worcestershire', '69'), ('Hounslow', '111')]
ROOT_DIR_PATH = os.path.abspath('..')

GHG_PROCESSED_PATH = os.path.join(ROOT_DIR_PATH, 'data/ground_truth_data/GHG_potential_sites.csv')

LOCAL_AUTHORITY_NAMES = ['Luton', 'Worcester', 'Hounslow', 'Portsmouth', 'Southampton', 'South Tyneside', 'Enfield', 'Halton', 'Barnet']

## General Functions

## AADT Data Loading and Pre-Processing

### Load Data

In [25]:
df_luton = pd.read_csv('https://storage.googleapis.com/dft-statistics/road-traffic/downloads/aadfbydirection/local_authority_id/dft_aadfbydirection_local_authority_id_120.csv')
df_worcester = pd.read_csv('https://storage.googleapis.com/dft-statistics/road-traffic/downloads/aadfbydirection/local_authority_id/dft_aadfbydirection_local_authority_id_69.csv')
df_hounslow = pd.read_csv('https://storage.googleapis.com/dft-statistics/road-traffic/downloads/aadfbydirection/local_authority_id/dft_aadfbydirection_local_authority_id_111.csv')
df_portsmouth = pd.read_csv('https://storage.googleapis.com/dft-statistics/road-traffic/downloads/aadfbydirection/local_authority_id/dft_aadfbydirection_local_authority_id_82.csv')
df_southampton = pd.read_csv('https://storage.googleapis.com/dft-statistics/road-traffic/downloads/aadfbydirection/local_authority_id/dft_aadfbydirection_local_authority_id_137.csv')
df_south_tyneside = pd.read_csv('https://storage.googleapis.com/dft-statistics/road-traffic/downloads/aadfbydirection/local_authority_id/dft_aadfbydirection_local_authority_id_166.csv')
df_enfield = pd.read_csv('https://storage.googleapis.com/dft-statistics/road-traffic/downloads/aadfbydirection/local_authority_id/dft_aadfbydirection_local_authority_id_121.csv')
df_halton = pd.read_csv('https://storage.googleapis.com/dft-statistics/road-traffic/downloads/aadfbydirection/local_authority_id/dft_aadfbydirection_local_authority_id_156.csv')
df_barnet = pd.read_csv('https://storage.googleapis.com/dft-statistics/road-traffic/downloads/aadfbydirection/local_authority_id/dft_aadfbydirection_local_authority_id_57.csv')


df_aadt_list = [df_luton, df_worcester, df_hounslow, df_portsmouth, df_south_tyneside, df_south_tyneside, df_enfield, df_halton, df_barnet]

print(df_luton.columns)

print(df_luton['count_point_id'].unique())

df_luton.head()

Index(['count_point_id', 'year', 'region_id', 'region_name',
       'local_authority_id', 'local_authority_name', 'road_name', 'road_type',
       'start_junction_road_name', 'end_junction_road_name', 'easting',
       'northing', 'latitude', 'longitude', 'link_length_km',
       'link_length_miles', 'estimation_method', 'estimation_method_detailed',
       'direction_of_travel', 'pedal_cycles', 'two_wheeled_motor_vehicles',
       'cars_and_taxis', 'buses_and_coaches', 'lgvs', 'hgvs_2_rigid_axle',
       'hgvs_3_rigid_axle', 'hgvs_4_or_more_rigid_axle',
       'hgvs_3_or_4_articulated_axle', 'hgvs_5_articulated_axle',
       'hgvs_6_articulated_axle', 'all_hgvs', 'all_motor_vehicles'],
      dtype='object')
[ 18221  27247  28472   6176  38079  48450  48057  73041  73040  73632
  77396  80935  99950  80936  80937 941719 941721 941688 941702 941713
  81081 941705 941700 941691  81080 941693 941708  81315  81457 951632
  17959 951634  77397 941699 941714 941716  84098  73039  73042  8145

,count_point_id,year,region_id,region_name,local_authority_id,local_authority_name,road_name,road_type,start_junction_road_name,end_junction_road_name,...,buses_and_coaches,lgvs,hgvs_2_rigid_axle,hgvs_3_rigid_axle,hgvs_4_or_more_rigid_axle,hgvs_3_or_4_articulated_axle,hgvs_5_articulated_axle,hgvs_6_articulated_axle,all_hgvs,all_motor_vehicles
0,18221,2014,7,East of England,120,Luton,A505,Major,Eaton Green Road roundabout,A5228,...,38,1099,171,57,23,11,14,43,318,8711
1,18221,2014,7,East of England,120,Luton,A505,Major,Eaton Green Road roundabout,A5228,...,35,1091,188,41,37,13,15,48,341,9314
2,27247,2014,7,East of England,120,Luton,A505,Major,Castle St roundabout,A6,...,124,1525,142,22,24,3,18,11,220,14023
3,27247,2014,7,East of England,120,Luton,A505,Major,Castle St roundabout,A6,...,95,1512,142,13,29,8,8,14,215,14091
4,28472,2014,7,East of England,120,Luton,A505,Major,Kimpton Road roundabout,Eaton Green Road roundabout,...,93,1429,118,16,31,10,26,26,228,11015


### Keep only Motorways and Remove unwanted Columns

In [26]:
for i in range(len(df_aadt_list)):
    df = df_aadt_list[i]
    df_motorways = df[df['road_name'].str.contains('M')]
    df_2005_onwards = df_motorways[df_motorways['year'] >= 2005]  
    df_2005_onwards =  df_2005_onwards[df_2005_onwards['year'] != 2021]  
    df_aadt_list[i] = df_2005_onwards.drop(UNWANTED_COLS, axis=1)

df_aadt_list[0]

,count_point_id,year,local_authority_id,local_authority_name,road_name,latitude,longitude,link_length_km,direction_of_travel,all_motor_vehicles
84,73039,2014,120,Luton,M1,51.902050,-0.475252,1.9,N,64166
85,73039,2014,120,Luton,M1,51.902050,-0.475252,1.9,S,57061
92,73042,2014,120,Luton,M1,51.890731,-0.469096,0.7,N,73265
93,73042,2014,120,Luton,M1,51.890731,-0.469096,0.7,S,61396
144,73042,2015,120,Luton,M1,51.890731,-0.469096,0.7,N,63750
...,...,...,...,...,...,...,...,...,...,...
1583,73042,2020,120,Luton,M1,51.890729,-0.469096,0.7,S,50240
1608,73039,2020,120,Luton,M1,51.902049,-0.475252,1.9,N,62896
1609,73039,2020,120,Luton,M1,51.902049,-0.475252,1.9,S,67612
1639,73039,2019,120,Luton,M1,51.902049,-0.475252,1.9,N,81098


### Groupby year and/or count point id

In [27]:
for i in range(len(df_aadt_list)):
    df = df_aadt_list[i]
    #grouped_df = df.groupby(by=['count_point_id', 'year']).mean()
    local_authority_name = df.iloc[0]['local_authority_name']
    grouped_df = df.groupby(by=['year']).median()
    grouped_df['Calendar Year'] = grouped_df.index
    grouped_df = grouped_df.reset_index()
    grouped_df['Local Authority'] = local_authority_name
    df_aadt_list[i] = grouped_df

print(len(df_aadt_list))
df_aadt_list[0]

9


/tmp/ipykernel_393/2068299700.py:5: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/tmp/ipykernel_393/2068299700.py:5: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/tmp/ipykernel_393/2068299700.py:5: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/tmp/ipykernel_393/2068299700.py:5: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will de

,year,count_point_id,local_authority_id,latitude,longitude,link_length_km,all_motor_vehicles,Calendar Year,Local Authority
0,2005,73040.5,120.0,51.896391,-0.472174,1.3,57600.0,2005,Luton
1,2006,73040.5,120.0,51.896391,-0.472174,1.3,48583.5,2006,Luton
2,2007,73040.5,120.0,51.896391,-0.472174,1.3,50652.5,2007,Luton
3,2008,73040.5,120.0,51.896391,-0.472174,1.3,53492.5,2008,Luton
4,2009,73040.5,120.0,51.896391,-0.472174,1.3,49612.5,2009,Luton
5,2010,73040.5,120.0,51.896391,-0.472174,1.3,54367.0,2010,Luton
6,2011,73040.5,120.0,51.896391,-0.472174,1.3,54802.5,2011,Luton
7,2012,73040.5,120.0,51.896391,-0.472174,1.3,56972.5,2012,Luton
8,2013,73040.5,120.0,51.896391,-0.472174,1.3,61684.0,2013,Luton
9,2014,73040.5,120.0,51.896391,-0.472174,1.3,62781.0,2014,Luton


## GHG Emissions Data Loading and Pre-Processing

In [28]:
df_ghg = pd.read_csv(GHG_PROCESSED_PATH, index_col=0)

df_ghg

,Local Authority,Calendar Year,Annual Territorial emissions (kt CO2e)
32,Barnet,2005,53.988609
33,Barnet,2006,56.822203
34,Barnet,2007,53.186986
35,Barnet,2008,59.597040
36,Barnet,2009,57.067457
...,...,...,...
2787,Worcester,2016,1.644139
2788,Worcester,2017,1.541270
2789,Worcester,2018,1.487991
2790,Worcester,2019,1.476269


In [29]:
df_ghg = df_ghg[df_ghg['Local Authority'].isin(LOCAL_AUTHORITY_NAMES)]

df_ghg_list = [d for _, d in df_ghg.groupby(['Local Authority'])]

len(df_ghg_list)

/tmp/ipykernel_393/4016481199.py:3: FutureWarning:

In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.



9

### AADT and GHG Emissions

In [30]:
merged_dfs_list = []

for i, aadt_df in enumerate(df_aadt_list):
    for j, ghg_df in enumerate(df_ghg_list):
        if aadt_df.iloc[0]['Local Authority'] == ghg_df.iloc[0]['Local Authority']:
            #print("AADT DF \n \n", aadt_df)
            #print("GHG DF \n \n", ghg_df)

            print(len(aadt_df))
            print(len(ghg_df))
            merged_df = aadt_df.merge(ghg_df, on=['Calendar Year', 'Local Authority'])
            merged_dfs_list.append(merged_df)

merged_dfs_list[0]

16
16
16
16
16
16
6
16
6
16
16
16
16
16
16
16


,year,count_point_id,local_authority_id,latitude,longitude,link_length_km,all_motor_vehicles,Calendar Year,Local Authority,Annual Territorial emissions (kt CO2e)
0,2005,73040.5,120.0,51.896391,-0.472174,1.3,57600.0,2005,Luton,31.812879
1,2006,73040.5,120.0,51.896391,-0.472174,1.3,48583.5,2006,Luton,25.993749
2,2007,73040.5,120.0,51.896391,-0.472174,1.3,50652.5,2007,Luton,27.204528
3,2008,73040.5,120.0,51.896391,-0.472174,1.3,53492.5,2008,Luton,27.863433
4,2009,73040.5,120.0,51.896391,-0.472174,1.3,49612.5,2009,Luton,24.754068
5,2010,73040.5,120.0,51.896391,-0.472174,1.3,54367.0,2010,Luton,27.443241
6,2011,73040.5,120.0,51.896391,-0.472174,1.3,54802.5,2011,Luton,27.513462
7,2012,73040.5,120.0,51.896391,-0.472174,1.3,56972.5,2012,Luton,28.448692
8,2013,73040.5,120.0,51.896391,-0.472174,1.3,61684.0,2013,Luton,29.845218
9,2014,73040.5,120.0,51.896391,-0.472174,1.3,62781.0,2014,Luton,30.298551


## Visualisation

### AADT

In [36]:
fig = go.Figure()

for i, df in enumerate(df_aadt_list):
    fig = fig.add_trace(go.Scatter(x = df['Calendar Year'], y=df['all_motor_vehicles'], name=df.iloc[0]['Local Authority']))

fig.update_xaxes(title='Year')
fig.update_yaxes(title='AADT')

fig.show()

### GHG Emissions

In [47]:
fig = go.Figure()

for i, df in enumerate(df_ghg_list):
    fig = fig.add_trace(go.Scatter(x = df['Calendar Year'], y=df['Annual Territorial emissions (kt CO2e)'], name=df.iloc[0]['Local Authority']))

fig.update_xaxes(title='Year')
fig.update_yaxes(title='Annual Territorial emissions (kt CO2e)')

fig.show()

### AADT and GHG Emissions

In [48]:
fig = go.Figure()

for i, df in enumerate(merged_dfs_list):
    fig = fig.add_trace(go.Scatter(x = df['all_motor_vehicles'], y=df['Annual Territorial emissions (kt CO2e)'], mode='markers', name=df.iloc[0]['Local Authority']))

fig.update_xaxes(title='AADT')
fig.update_yaxes(title='Annual Territorial emissions (kt CO2e)')

fig.update_layout(
    autosize=False,
    width=1500,
    height=1000,)
fig.show()